# Range/Doppler Animation

This notebook calculates the CAF and renders an animated plot over a specified amount of (world-)time.

## Setup

In [ ]:
%matplotlib widget

import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation

import multiprocessing as mp

import collections

import typing
import numpy.typing as npt

import sys
sys.path.append('../')
import pluto_sdr_pr.ioutils
import pluto_sdr_pr.processing

## Constants

In [ ]:
c = 3e8 # in meters per second
Fs = 2.88e6 # in Hertz
fc = 625e6 # in Hertz

## Plot Parameters

In [ ]:
cpi = 1 # in seconds
num_samples = int(cpi * Fs)
time_window = 100 # in seconds
num_frames = int(time_window // cpi)
max_distance = int(50e3) # in meters
max_delay = int(max_distance / c * Fs)
max_speed = 280 # in meters per second
max_doppler = int(max_speed * cpi * fc / c)

## Rendering

In [ ]:
%%time

def calculate_absolute_ambiguity(file_path: str, frame: int):
    with open(file_path, 'rb') as fid:
        n = pluto_sdr_pr.ioutils.read_samples(fid, num_samples, frame * num_samples)
        amb = pluto_sdr_pr.processing.fast_ambiguity(max_delay, 2 * max_doppler + 1, n, n).T
        return {frame: np.abs(amb)}

def render_frame(frame_stack: np.ndarray, frame: int, num_frames: int, cpi: float) -> typing.Tuple[mpl.artist.Artist, mpl.artist.Artist]:
    return (plt.imshow(frame_stack[:, :, frame], interpolation='nearest', aspect='auto'),
            plt.text(0, -1.5, f"frame: {frame + 1}/{num_frames} {frame * cpi:.1f} sec"))

plt.ioff()

fig, ax = plt.subplots(figsize=(19.2, 10.8))
ax.set_title("Range/Doppler Map")
ax.set_xlabel("bistatic range [km]")
ax.set_ylabel("bistatic velocity [m/s]")
xticks = np.linspace(0, 50e3, 6, endpoint=True)
ax.set_xticks(xticks / c * Fs)
ax.set_xticklabels(map(lambda x: f"{x / 1e3:.0f} km", xticks))

yticks = np.linspace(-max_speed, max_speed, 15, endpoint=True)
ax.set_yticks((yticks + max_speed) * cpi * fc / c)
ax.set_yticklabels(map(lambda y: f"{y:.0f} m/s", yticks))

file_path = '../data/gqrx_20210326_161137_625000000_2880000_fc_flugplatz_longdipole.raw'

with mp.Pool() as pool:
    frames = pool.starmap(calculate_absolute_ambiguity, [(file_path, frame) for frame in range(num_frames)])

frame_chain = collections.ChainMap(*frames)
frame_stack = np.stack(tuple((frame_chain[frame] for frame in sorted(frame_chain))), axis=2)
frame_stack = 10 * np.log10(np.divide(frame_stack, np.max(frame_stack)))

anim = animation.ArtistAnimation(fig, [render_frame(frame_stack, frame, num_frames, cpi) for frame in range(frame_stack.shape[2])], interval=cpi * 1e3)
anim.save("range-doppler.mp4", writer=animation.FFMpegWriter(fps=1 / cpi, codec='png'), dpi=100)
del anim, fig, ax

plt.ion()